In [4]:
import collections
import itertools as it
import json
from pathlib import Path
import random
import subprocess

import jsonlines
import numpy as np

In [16]:
# set the file path
root_path = "/home/mila/g/gagnonju/Marg-Li-CoT/with_trl/libs_data/arithmetic/outputs/train_scratch/"

N = 2

paths = Path(root_path).glob("train_scratch.*.jsonl")

def extract_number(path):
    return int(path.name.split(".")[1])

sorted_paths = sorted(paths, key=extract_number)

few_shot_per_number = collections.defaultdict(list)
for file_path in sorted_paths:
    number = extract_number(file_path)
    output_list = few_shot_per_number[number]
    num_lines = int(subprocess.check_output(['wc', '-l', file_path]).split()[0])
    idx = sorted(np.random.permutation(num_lines)[:N])

    # Don't keep the samples in memory
    with jsonlines.open(file_path) as reader:
        reader_it = enumerate(reader)
        for idx_requested in idx:
            idx_given, value = next(reader_it)
            while idx_given != idx_requested:
                idx_given, value = next(reader_it)

            assert value["num_digits"] == number, (
                f"number {value['num_digits']} != {number}")
            
            output_list.append(value)

for num_digits, v in few_shot_per_number.items():
    assert len(v) == N, f"{num_digits = }, len(v) = {len(v)} != {N}"
    print(f"{num_digits = }, {len(v) = }")


output = list(it.chain.from_iterable(few_shot_per_number.values()))
random.shuffle(output)

print(json.dumps(output, indent=4))

num_digits = 1, len(v) = 2
num_digits = 2, len(v) = 2
num_digits = 3, len(v) = 2
num_digits = 4, len(v) = 2
num_digits = 5, len(v) = 2
[
    {
        "input": "7 2 6 4 + 5 7 0 2",
        "answer": "1 2 9 6 6",
        "num_digits": 4,
        "scratchpad": "<scratch>\n7 2 6 4 + 5 7 0 2 , C: 0\n6 + 0 , 6 C: 0\n2 + 7 , 6 6 C: 0\n7 + 5 , 9 6 6 C: 0\n, 2 9 6 6 C: 1\n1 2 9 6 6\n</scratch>"
    },
    {
        "input": "8 + 4",
        "answer": "1 2",
        "num_digits": 1,
        "scratchpad": "<scratch>\n8 + 4 , C: 0\n, 2 C: 1\n1 2\n</scratch>"
    },
    {
        "input": "3 3 + 5 5",
        "answer": "8 8",
        "num_digits": 2,
        "scratchpad": "<scratch>\n3 3 + 5 5 , C: 0\n3 + 5 , 8 C: 0\n, 8 8 C: 0\n0 8 8\n</scratch>"
    },
    {
        "input": "8 2 9 + 5 6 3",
        "answer": "1 3 9 2",
        "num_digits": 3,
        "scratchpad": "<scratch>\n8 2 9 + 5 6 3 , C: 0\n2 + 6 , 2 C: 1\n8 + 5 , 9 2 C: 0\n, 3 9 2 C: 1\n1 3 9 2\n</scratch>"
    },
    {
        "input"

[10, 10, 10, 10, 10]